 
<h1 align="center">🚀 PVXT - QA with Citations</h1>

----

## 🛠️ TLDR : Build 5 vision and text use cases with Claude 3 Sonnet on Amazon Bedrock:


- **1. Q&A with an AWS Architecture diagram** :
Learn how to integrate Claude 3 Sonnet into your AWS architecture to enhance AI capabilities. 🏗️

- **2 Automate Terraform script from AWS architecture diagram**:
Learn how to build a Terraform deployment with AWS architecture images using Claude 3 Sonnet. 🛠️

- **3. Side by Side Image Comparison**:
Discover the power of image comparison using Claude 3 Sonnet on Amazon Bedrock. 🖼️

- **4. Image Transcription**:
Explore the capabilities of image transcription with Claude 3 Sonnet on Amazon Bedrock. 📝

- **5. Streaming text reponse**:
Understand how to use Claude 3 Sonnet for streaming text responses. 📜

---

### Introduction to Claude 3

Anthropic unveils the Claude 3 model family, featuring Haiku, Sonnet, and Opus. 🌐

These models redefine AI capabilities, catering to diverse needs. Claude 3 Sonnet is available for immediate use on Amazon Bedrock, with Claude 3 Opus and Claude 3 Haiku coming soon. 💻

Claude 3 Sonnet strikes the ideal balance between intelligence and speed—particularly for enterprise workloads. It delivers robust performance at a lower cost compared to its peers and is engineered for high endurance in large-scale AI deployments. Moreover, it introduces image-to-text vision capabilities! 📸

**Potential uses**:
- Data processing: RAG or search & retrieval over vast amounts of knowledge 📚
- Sales: product recommendations, forecasting, targeted marketing 📈
- Time-saving tasks: code generation, quality control, parse text from images ⏰

**Differentiator**: More affordable than other models with similar intelligence; better for scale. 💰

**Key features**:

- 📊 **Context Window:** 200,000 tokens

- ⚡ **Speed:** 2x faster than Claude 2 and Claude 2.1

- 👁️ **Vision Capabilities:** Handles a wide range of visual formats, including photos, charts, graphs, and technical diagrams.

- 🚫 **Refusals:** Claude 3 models (including Sonnet) significantly reduce unnecessary refusals compared to previous generations.

- 🛡️ **Safety Level:** Maintains AI Safety Level 2 (ASL-2) per the Responsible Scaling Policy.

- 🚀 **Model Speed Comparison:** Sonnet is 2x faster than Claude 2 and Claude 2.1, excelling at tasks demanding rapid responses.


### Pre-requisites:

- **AWS Account**: To use Amazon Bedrock and deploy the use cases.
- **AWS Credentials**: Make sure your credientials are configured in your environment. More details [here](https://handsonaws.com/blog/advanced-awscli-notebooks-iam-s3-cross-account/)

If you have a AWS profile configured in your environment, you can use the following magic commands to set the profile and rehash the environment variables:

```bash 
%env AWS_PROFILE = your_profile
%rehashx
%automagic 1
```
- **Amazon Bedrock permissions** : Amazon Bedrock IAM permissions to run this notebook. More details [here](https://docs.aws.amazon.com/bedrock/latest/userguide/iam-permissions.html)

Enable AWS IAM permissions for Bedrock
The AWS identity you assume from your notebook environment (which is the Studio/notebook Execution Role from SageMaker, or could be a role or IAM User for self-managed notebooks), must have sufficient AWS IAM permissions to call the Amazon Bedrock service.

To grant Bedrock access to your identity, you can:

Open the AWS IAM Console
Find your Role (if using SageMaker or otherwise assuming an IAM Role), or else User
Select Add Permissions > Create Inline Policy to attach new inline permissions, open the JSON editor and paste in the below example policy:


```json 
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockFullAccess",
            "Effect": "Allow",
            "Action": ["bedrock:*"],
            "Resource": "*"
        }
    ]
}
```



### Use case 1: Q&A with an AWS Architecture diagram using Claude 3 Sonnet on Amazon Bedrock

In this use case, we will explore how to integrate Claude 3 Sonnet to help you answer questions about your AWS architecture. This can be particularly useful for large-scale deployments, where it can be challenging to keep track of all the moving parts.




In [ ]:
# Install latest version of boto3
!pip --quiet install boto3==1.33.2 termcolor matplotlib ipython


In [ ]:
import base64, re,json
import boto3
from termcolor import colored
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
from IPython.display import display, Image


In [ ]:
def image_to_base64(image_path):
    with open(image_path, 'rb') as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def display_image(image_path):
    with open(image_path, "rb") as image_file:
        display(Image(data=image_file.read()))

In [ ]:
display_image("images/architecture.png")

The Claude 3 family of models comes with new vision capabilities that allow Claude to understand and analyze images, opening up exciting possibilities for multimodal interaction. With Claude, you can now provide both text and image inputs to enrich your conversations and enable powerful new use cases.


To utilize images when making an API request, you can provide images to Claude as a ``base64-encoded`` image in image content blocks. Here is simple example in Python showing how to include a base64-encoded image in a Messages API request:

```python
messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": image1_media_type,
                        "data": image1_data,
                    },
                },
                {
                    "type": "text",
                    "text": "Describe this image."
                }
            ],
        }
    ],
```

Supported image formats are ``JPEG, PNG, GIF, and WebP``. See Messages [API examples](https://docs.anthropic.com/claude/reference/messages_post) for more example code and parameter details.

Below we defined two funtions to help you get started with image processing using Claude 3 Sonnet on Amazon Bedrock:

- ``post_process_answer()`` : This function extracts answer from the Bedrock API response. A best pratice frome [Claude prompt engineering](https://docs.anthropic.com/claude/docs/prefill-claudes-response) is to output the answer inside <answer></answer> XML tags. This function will extract the answer from the API response.
- ``generate_vision_answer()`` : This function will generate the API request to send the image to the Claude 3 Sonnet model to Amazon Bedrock. 




In [ ]:
def post_process_answer(response:str)->str:
    """
    Extracts the answer from the given response string.

    Args:
        response (str): The response string containing the answer.

    Returns:
        str: The extracted answer.
    """
    answer = re.findall(r'<answer>(.*?)</answer>', response, re.DOTALL)
    return answer[0]

def generate_vision_answer(bedrock_rt:boto3.client,messages:list, model_id:str, claude_config:dict,system_prompt:str):
    """
    Generates a vision answer using the specified model and configuration.
    
    Parameters:
    - bedrock_rt (boto3.client): The Bedrock runtime client.
    - messages (list): A list of messages.
    - model_id (str): The ID of the model to use.
    - claude_config (dict): The configuration for Claude.
    - system_prompt (str): The system prompt.
    
    Returns:
    - str: The formatted response.
    """
    
    body={'messages': [messages],**claude_config, "system": system_prompt}
    
    response = bedrock_rt.invoke_model(modelId=model_id, body=json.dumps(body))   
    response = json.loads(response['body'].read().decode('utf-8'))
    formated_response= post_process_answer(response['content'][0]['text'])
    
    return formated_response

In this use case, the goal is to ask a question about the AWS architecture and provide an image of the architecture. The image will be sent to the Claude 3 Sonnet model to generate a response. The response will be post-processed to extract the answer.

To perform this task, we need a **system prompt**. This is a way of providing context and instructions to Claude, such as specifying a particular goal or role. See our guide to system prompts.

An example system prompt for this use case could be:

```
You have perfect vision and pay great attention to detail which makes you an expert at answering architecture diagram question
```

In [ ]:
%%time

# Create a bedrock runtime client in us-west-2
bedrock_rt = boto3.client("bedrock-runtime", region_name="us-east-1")



# Model id and claude config
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
claude_config = {
    'max_tokens': 1000, 
    'temperature': 0, 
    'anthropic_version': '',  
    'top_p': 1, 
    'stop_sequences': ['Human:']
}

# Create prompt and system prompt
system_prompt= "You have perfect vision and pay great attention to detail which makes you an expert at answering architecture diagram question. Answer question in <question></question> tags. Before answer, think step by step in <thinking> tags and analyze every part of the diagram."

#Create a prompt with the question
prompt = f"<question>Explain why Amazon Gamelift is usefull on the diagram </question>. Answer must be a numbered list in a small paragraph inside <answer></answer> tag."

# Create message with the prompt and the base64 encoded image
messages={"role": "user", "content": [
{
        "type": "image",
        "source": {
        "type": "base64",
        "media_type": "image/jpeg",
        "data": image_to_base64("images/architecture.png"),
        }
},
{"type": "text", "text": prompt}
]}

# Generate answer
answer=generate_vision_answer(bedrock_rt, messages, model_id, claude_config, system_prompt)    
print(colored(answer, "green"))


### Use Case 2: Automate Terraform script from AWS architecture diagram using Claude 3 Sonnet on Amazon Bedrock

In this use case, we will explore how to use Claude 3 Sonnet to generate a Terraform script from an AWS architecture image. This can be particularly useful for automating the deployment of AWS infrastructure. It will greatly reduce the time and effort required to create Terraform scripts from AWS architecture images.


Here is a simple serveless architectiure diagram that we will use as an example:


In [ ]:
display_image("images/serverless.png")

In [ ]:
system_prompt=""
messages = {    
    "role": "user",
    "content": [
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/jpeg",
                "data": image_to_base64("images/serverless.png")
            }
        },
        {
            "type": "text",
            "text": "Describe the architecture and code terraform script to deploy it, answer inside <answer></answer> tags."
        }
    ]
}

# Generate answer
answer= generate_vision_answer(bedrock_rt, messages, model_id, claude_config, system_prompt="")
print(colored(answer, "green"))

### Use case 3:  Side by side image comparaison using Claude 3 Sonnet on Amazon Bedrock

In this use case, we will explore how to use Claude 3 Sonnet to compare two images side by side. This can be particularly useful for quality control and identifying differences between two images.

#### Let's play a game!

Can you spot the differences between the two images below?


In [ ]:
%matplotlib inline

local_images = ["images/image_1.png", "images/image_2.png"]

# figure size in inches optional
rcParams['figure.figsize'] = 11 ,8

# read images
img_A = mpimg.imread(local_images[0])
img_B = mpimg.imread(local_images[1])

# display images
fig, ax = plt.subplots(1,2)
ax[0].imshow(img_A)
ax[1].imshow(img_B)

In [ ]:
%%time
import base64

list_images_base64 = []
local_images = ["images/image_1.png", "images/image_2.png"]


system_prompt="You have perfect vision and pay great attention to detail which makes you an expert at finding differences between images. Answer question in <question></question> tags, answer must be inside <answer></answer> tag. Before answer,  Think step by step in <thinking> tags and analyze every part of the image."
prompt="List all difference between the two pictures ?"
messages={"role": "user", "content": [
        {
                    "type": "text",
                    "text": "Image 1:"
            },
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/jpeg",
                "data": image_to_base64(local_images[0]),
            }
        },
        {
            "type": "text",
            "text": "Image 2:"
        },
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/jpeg",
                "data": image_to_base64(local_images[1]),
            }
        },
        {"type": "text", "text": prompt}
    ]}
# Generate answer
answer= generate_vision_answer(bedrock_rt, messages, model_id, claude_config, system_prompt)    
print(colored(answer, "green"))

### Use Case 4: Image Transcription with Claude 3 Sonnet on Amazon Bedrock 

In this use case, we will explore how to use Claude 3 Sonnet to transcribe text from an image. This can be particularly useful for extracting text from:

1. Images
2. Charts
3. Diagram
4. Technical Drawings


In [ ]:
display_image("images/organigram.jpeg")

In [ ]:
messages = {
        "role": 'user',
        "content": [
            {"type": "image", "source": {"type": "base64", "media_type": "image/jpeg", "data": image_to_base64("images/organigram.jpeg")}},
            {"type": "text", "text": "Turn this org chart into JSON indicating who reports to who, answer inside <answer></answer> tags."}
        ]
    }

# Generate answer
answer= generate_vision_answer(bedrock_rt, messages, model_id, claude_config, system_prompt="")
print(colored(answer, "green"))

### Use Case 5: Streaming Text with Claude 3 Sonnet on Amazon Bedrock

In this use case, we'll explore how to harness the power of Claude 3 Sonnet to stream text. This functionality proves particularly beneficial for:

- Real-time updates
- Live conversations
- Dynamic content generation
- Continuous text processing


In [ ]:
#Streaming with Bedrock 3


def stream_answer(bedrock_rt:boto3.client, model_id:str, claude_config:dict, prompt:str):
    stream_answer={"role": "user", "content": prompt}
    
    body={'messages': [stream_answer],**claude_config}
    response = bedrock_rt.invoke_model_with_response_stream(modelId=model_id, body=json.dumps(body))
    response_stream = response.get('body')
    for event in response_stream or []:
        chunk_data = event.get('chunk')
        if chunk_data:
            chunk_obj = json.loads(chunk_data.get('bytes').decode())
            delta = chunk_obj.get('delta', {})
            text_content = delta.get('text', "")
            yield str(text_content)

In [ ]:
prompt="""

Human: I'm going to give you a document. Then I'm going to ask you a question about it. I'd like you to first write down exact quotes of parts of the document that would help answer the question, and then I'd like you to answer the question using facts from the quoted content. Here is the document:

Playback Experience Tech - Product and Program Team 

Team Overview
Playback brings the technologies, tools, and operations to stream Live, Linear, and VOD content to all devices globally. Our infrastructure brings flawless, high-quality streaming while using the fewest bits possible. We aim to deliver a better experience than broadcast. We innovate to efficiently manage our global scale across CDNs, ISPs, every sort of home network, and 10K+ device models. Video playback includes acquiring the video (and audio) content from partners, Encoding (compression) of the video, policy enforcement to meet locale and studio requirements, measuring the quality of the experience (QoE), vending of all playback resources, managing the storage of the content, delivery of the video fragments over CDN / ISP networks to the customer's device and the playing back of the video on the customer's device including UI control of the video player. Playback also manages our operational team responsible for monitoring and mitigating our video delivery solution, including VOD, Live Events, and Linear channels. We also constructed the Analytics system that measures our video playback, including an AI-based monitoring and alerting system to identify issues in real-time. We can then root causes and swiftly mitigate these on a global scale. The data generated by the Playback system is the source of truth for video consumption and usage. We share the data in a privacy-safe manner across all stakeholders. In addition, our infrastructure optimizes video delivery across the network in real-time so viewer always get the best possible quality that their network and device is capable of supporting. Finally, we provide video playback to our tenants, such as Freevee and Music, bringing economies of scale and innovation to Amazon video properties.

We split our team into the following product and program clusters:

Encoding and Picture Quality (PQ)
Live Streaming - Live Events and Linear
Playback Services and Video Delivery
Video Player, Playback Experience, Video Analytics, Optimization
Live and Linear Program Management
Quality of the Experience (QoE) and Picture Quality (PQ) Program Management
Recent Updates
See our new Documentation Portal!

https://w.amazon.com/bin/download/Amazon_Video/PV_Playback/Product_and_Program/New_Home/WebHome/docs.png

Mission
Prime Video Playback technologies are the benchmark for video streaming, providing the best viewing experience for our customers globally. Amazon's best-in-class library (VOD and Live) will always be the preferred choice for customer entertainment anywhere and on any device. Our applications and services are secure, protecting content (Amazon and partners), customer information, and our business. In the future, our solution will allow us to support >500MM active customers worldwide in the most cost-effective way possible while enabling multi-tenants to promote their video business expansion.

Our mission is to build and operate the most innovative video streaming technology that provides the best streaming for VOD and Live across all devices globally. We measure our success by delivering the earth's most customer-centric streaming globally. We built a multi-tenant stack used and extended by other organizations in Amazon. We secure our digital content to maintain trust with our partners.

Our Tenets (Unless You Know Better...)
We willingly adopt Audio/Video (A/V) streaming standards and implement them at scale. We invent new A/V tech without constraints on behalf of our customers, influencing original equipment manufacturers (OEM), content delivery networks (CDN), internet service providers (ISP), industry partners, and standards bodies to adopt our proposals and innovate with us. We do not merely adopt emerging tech. We strive to invent them.
We operate our services by continually improving 1) reliability and availability, 2) video experience and picture quality, and 3) new customer-facing capabilities. When we have to make tradeoffs, we prioritize in the order listed.
We design our services with multi-tenancy and decoupling in mind. Our stakeholders shouldn't rely on us to deliver for their customers. When we slow them down, we'll take a step back and assess how to enable them to move fast and increase velocity.
Delivering video at scale reliably is a complex problem involving complicated tech and expertise. We are biased towards developing and operating these technologies in-house and fall back to third-party (3P) technology when we see an opportunity to accelerate customer benefit.
We will continuously lower the cost of bits-on-the-wire while ensuring we don't compromise the perceived customer viewing experience.
We launch new products and services with scalable, automated processes. Any exception is a highly considered decision with a swift plan to close the gaps. We will not launch a new feature directly on high-profile content. We will instead iteratively deploy on low-impact and low-risk content. When we have high confidence, we will roll it out for high-profile content.

Team Overview
Our team is responsible for the sound development of the playback solution. The team assesses, defines, prioritizes, and supports the development of solutions for the video playback organization. We prioritizes our roadmap based on our customer obsession - across the video playback organization. The team defines and captures the strategic vision, establishes the roadmap, defines the requirements, and works closely with engineering teams to execute on our vision. The groups encompass the entire video pipeline, including Video Players (Android, Web, iOS, Living Room), Encoding, Live Playback, Video Delivery, Playback Insights, Content Access, Playback Services, and Playback Experience. We drive Product Mechanics, focusing on a well-understood cadence, transparency, accountability, and customer value. We write 3-year Vision documents, PRFAQs, BRDs, Product Goals and metrics, and requirements documents outlining the customer experience. We jointly work with engineering teams to define our technical approach and workback plans. We are customer-obsessed and partner with all stakeholders to ensure we deliver the best experience for our customers. We synchronize with the industry to influence their roadmaps and technology decisions to align with Prime Video's vision. We determine long-term leapfrogging ideas for the future of digital video consumption, focusing the team on the execution of the next-generation playback technologies and experiences that delight our customers.

Success Metrics
Customer Experience & Satisfaction
We measure our performance holistically using several metrics for each critical aspect of our business. Our solutions aim to improve key business metrics such as 1) Hours Streamed (Absolute and CAGR), 2) Peak Concurrent Plays (Absolute and CAGR), or 3) Number of Monthly Active Users (MAU). We group Playback critical metrics into the following categories: 1) measure our ability to scale, 2) the Quality of the Experience (QoE) rendered to our viewer, 3) the perceptual Picture Quality (PQ) of our streaming, 4) the customer satisfaction driven by our video Playback capabilities, and 5) our Cost and Operational Efficiency of running the video infrastructure.

We measure our Scale and Availability with Peak Concurrent Plays (PCP) sessions (in Million/M), # Video Streams (in Billion/BN), # Viewing Hours Streamed without issues (in Billion/BN), # Countries, # Linear Stations, # Live Events, # Live Events on Linear (LEL), # VOD Titles (in MM titles), # of Broadcast Hours for all Live Events, Total Bytes Delivered (in Petabyte/PB) and % of Traffic delivered with CloudFront
We assess our customer Quality of Experience (QoE) with % Time Spent Buffering, Time-to-First-Frame (TTFF), Zero Error Rate (ZER), Zero Buffer Rate (ZBR), Exit Before Video Start (EBVS), and Buffering Abandoned Plays (BAP). We strike a balance to optimize for these metrics, aiming to drive a "better than broadcast" experience given constraints such as available bandwidth, cap limitation, and cost
We assess our customer Picture Quality (PQ) by leveraging the % of HD and UHD fragments for suitable sessions. We optimize for High Dynamic Range (HDR) and Dolby Vision coverage. We also use subjective testing and golden eyes to assess our quality. We introduce new measurements with Video Multi-method Assessment Fusion (VMAF at P90, Amazon Quality Measurement, ITU-T REC P.1204) to improve our quantification in this area. These PQ metrics use mathematical models to compute the video quality automatically
We assess Customer Satisfaction when we launch new capabilities and value brought by our video experience - usability testing, NPS, or other proxies such as social metrics or press clipping. We evaluate customer engagement with playback capabilities - Reviews & feedback, Customer sentiment score, app store ratings for video playback
Finally, we obsess about our Cost and Operation Efficiency with KPIs such as Playback Cost Per 1000 Streams (CPS), Encode cost Per Hour (EPH), CDN cost, and per Linear station cost. We also measure our operational and monitoring efficiency by leveraging KPIs such as Operators per Live Events or Linear stations.
Finally, we aim to constantly improve the speed at which we deliver new capabilities to our customers. We strive for faster roadmap velocity and value brought to our customers and stakeholders. The product & program team must increase predictability with a sustained cadence understood by all. We reduce blockers reported and lower scope-cuts/scope-creep to meet deadlines and expectations. We support engineering in reducing counts of defects reported during delivery and better handling of quality levels. Finally, our team aims to reduce our churn by making our work enjoyable with a purpose for both the team and our stakeholders - continuously working on our connection scores and bringing the maximum value to our customers.

Product & Program Velocity
We aim to constantly improve the speed at which we deliver new capabilities to our customers. We strive for faster roadmap velocity and value brought to our customers and stakeholders. The product & program team must increase predictability with a sustained cadence understood by all. We reduce blockers reported and lower scope-cuts/scope-creep to meet deadlines and expectations. We support engineering in reducing counts of defects reported during delivery and better handling of quality levels. Finally, our team aims to reduce our churn by making our work enjoyable with a purpose for both the team and our stakeholders - continuously working on our connection scores and bringing the maximum value to our customers.

Leadership
Olivier Wellmann
wellmo@
Director
Director of Product and Program Management 
[3-year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [CPR/WBR], [KPI]
Video Playback Product Strategy across Amazon Business Cadence (3YP, OP, QPR)
Product Roadmap and Execution for Video Playback
Technical Product Management Leader
Technical Program Management Leader
Product Goals for Playback
Recruiting, coaching, and mentoring
Product and Program Escalation

Martha Cesari
cehmarth@
EA
Executive Assistant 
Manages Olivier's calendar

Video Innovation
Innovation focuses on rising the bar in Camera-to-Cornea customer experience though industry and stakeholder collaboration, whole product perspective, and get patents. Prime Video works with many external and internal partners from content producers to post houses, validation and analysis tools, encoding and packaging tools, CDNs, and consumer electronics companies. As new technologies and features constantly emerge, we need to focus on those that will provide customer value, and work with partners to bring them to market quickly, efficiently, and with high quality. Much is done bilaterally, delivering Prime Video first technologies like HDR10+. We also build upon many industry standards, and Prime Video contributes to standards that raise the bar for customers, like Filmmaker Mode.
While Prime Video is made by a huge number of teams and individuals all working on aspects of the service, Our customers receive Prime Video as a whole product. We innovate looking at the big picture for cross-team bar-raising efforts. Examples include recommending cameras and settings for Amazon Studios, delivering more consistent audio loudness, and providing customers optimum picture modes.

Ben Waggoneer
benwagg@
TPM
Principal Video Specialist 
Video Technology Leader and Subject Matter Expert
Prime Video Audio/Video Evangelist
Emerging Technology evaluation and proofs of concept,
player control over picture modes, new media technologies, VR, 3D, multi primary video
Quality Innovation, next-gen video (AV1, VVC AV2), 8K and UHFR (100 and 120 fps) initiatives, ambient light adaptation, dynamic metadata, heuristic development, lens-to-screen whole product quality and accuracy.
Audio Quality Innovation, including next gen spatial audio (MPEG-H, AC4, IAMF), loudness control, dialog clarity, and other audio enhancements
Streaming Product Architecture Improvements
Video Industry Partnerships, including UHD Alliance (UHDA), UHD Forum (UHDF), 8K Association, and SMPTE
Bidirectional innovation partnerships with studios and CE companies
Technology Patenting


Live Event & Linear, Live Operations & Monitoring, VOD Encoding & AV Quality
Live Events & Linear: The Live Event and Linear Product Teams owns the live events and linear product offerings for Playback. Our mission is to build and operate the world’s most innovative live video streaming technology stack, providing a customer-centric live streaming experience. Our desire is to set the bar for the industry and position PV as the top streaming service in the world by being significantly better than broadcast TV in terms of availability, quality, reliability, and scale for live streaming.
Live Operations & Monitoring: The Live Operations and Live Monitoring Product teams owns the operation and end-to-end monitoring of all Live events and Linear stations on Prime Video. The program ensures we can rapidly onboard and operate events and linear stations at scale, including automation (all processes from pre-event to post-event), best-in-class 24/7 monitoring (finding defects before customers tell us), and actively monitor and triage Live event issues to mitigate core playback issues in <3 Minutes. We measure success by the number of events and Linear stations we delivered without issues. We continuously drive operational efficiency to improve our ability to swiftly launch events and stations cost-effectively.
VOD Encoding & AV Quality: The VOD Encoding Product Team owns the generation of all VOD media (card insertion, ad insertion, and main title variants) and is responsible for all VOD picture and audio quality, content compression (and the associated globally distributed storage) using various encoding algorithms and packaging of the content to support the audio/video capabilities for 10K+ customer devices. This team is responsible for building the pipelines for the analysis, preprocessing, encoding, packaging, and encryption of video and audio streams, as well as delivering the highest quality consumer content by building sufficient guardrails in the systems and services. The Encoding team mission is to create the highest quality video, audio, captions and images possible for Prime Video customers worldwide.

Scott Mackenzie
scmacken@
Sr. Mgr Product
Playback Product Lead for Live Events, Linear, VOD Encoding, Video Quality Analysis and Live Operations Technology
Live Events Playback Product Leader
Linear Playback Product Leader
VOD Encoding Playback Product and Program Leader
Video Quality Analysis Playback Product Leader
Live Operations Technology Product Leader
[3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Playback Product Strategy (3YP, OP, QPR, MBR)
Playback Product Roadmap and Execution
Playback Product and Program Recruiting
Playback Product and Program Escalation
Playback Product and Program Owner
Heddy Kuhl

kuhheddy@
PMT
Senior Product Manager Tech for Live Events
Playback Live Event Product Leader
Playback Live Event [3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Live Sports Org Playback Partner
Playback Live Event Product Owner & Strategy
Playback Live Event Product Roadmap and Execution
Playback Live Event Capabilities (i.e. time-shifting, bitrate profiles, CX features)
Playback Live Signal Ingestion of Live Events and Concerts
Playback Live Event Onboarding, including transcoding, orchestration, content origin, and live monitoring
Playback Live Infrastructure, encoding, packaging and origin
Playback Live Encoding, a/v codecs and bitrate profiles
Partner Management for AWS Elemental Roadmap


Oleg Tsaregorodstev
oletsare@
PMT
Principal Product Manager Tech for Linear TV
Playback Linear Product Leader
[3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Playback Linear Product Strategy (3YP, OP, QPR, MBR)
Playback Linear Product Roadmap and Execution
Playback Linear Product Owner
Playback Linear Signal Ingestion
Playback Linear Infrastructure, including encoding, packaging and origin
Free Ad-supported Streaming TV (FAST)
Sports rights Ad-Hoc Aggregators (AHA)
Live Events on Linear
Linear Capabilities (e.g., time-shifting, multi-track audio)

Max Denton
maxdent@
PMT
Principal Product Manager Tech for VOD Encoding
Playback VOD Encoding Product Leader
[3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Playback VOD Encoding Product Strategy (3YP, OP, QPR, MBR)
Playback VOD Encoding Roadmap and Execution
[CPR/WPR], [Mechanics (MBR/QPR/Roadmap)], [Technical Design], [Cross Team Alignment]
Gen 4 Next Generation Encoding Pipeline Product Leader
Card Stitching Program Leader (End to End)
Encoding Latency On-Time Publishing Product Leader
Encoding Infrastructure Cost Out Product Leader
VOD Ads scaling
VOD AV1 roll out

Andrew Whittaker
awhittak@
TPM
Playback Sr. Technical Program Manager (TPM)
PV single threaded TPM leader for Card Stitching feature
VOD Encoding Card Stitching (CS) Program Leader
Standing in for PV single threaded PMT leader for Card Stitching feature.
TPM for Playback Encoding
[CPR/WPR], [Mechanics (MBR/QPR/Roadmap)], [Technical Design], [Cross Team Alignment]
Gen 4 Next Generation Encoding Pipeline Program
Latency On-Time Publishing Program
Infrastructure Cost Out Program
Ads encoding scaling
AV1 roll out
Team Mentoring

Joel Korpi
korpijk@
PMT
Senior Product Manager Tech for VOD Encoding, Picture Quality and Codecs
Currently on Leave of Absence, see scmacken@ for coverage
Playback VOD Encoding Product Leader
[3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Playback VOD Encoding Picture Quality and Codec Product Strategy (3YP, OP, QPR, MBR)
Playback VOD Encoding Picture Quality, Codec Roadmap and Execution
Playback VOD Codec strategy, including AV1/AV2 (Alliance for Open Media) and Versatile Video Coding (VVC/H.266)
Playback VOD R&D, including Cinema+ and TruColor
Partner Management for Dolby Laboratories

Playback Services and Video Delivery
This product area owns the end-to-end flow from when the customer hits play until their playback session is complete.  We build the integration point between backend services and customer-facing video players to 1) deliver the correct stream entitlement, 2) optimize manifests for the customer and device, and 3) enforce the concurrency limits set by subscription policies. It also covers services that protect content (DRM, Geo Services, VPN abuse) and tracks user sessions to ensure they can resume where they left off on any device. Our video delivery services takes the encoded bits from content origins and delivers the asset to our customer's devices over the internet. We continuously ensure the availability and network capacity to bring millions of concurrent streams to customers reliably across CDNs, ISPs, mobile networks, and home networks. We build with AWS and 3P providers, caching capacity and content as close to the customer as possible. Using real-time data and ML optimization, we determine the most cost-effective network path for a customer session while providing the highest quality stream.

Eric Boyd
byde@
Sr. Mgr Product
Sr. Manager Product, Playback Services and Video Delivery
Product lead for video delivery and play start services that deliver playable streams to customers world-wide
Product Strategy and Roadmap Execution (3YP, OP, MBR, QPR)
CDN edge authorization (Edge tokenization)
Granular Session Routing and optimized CDN URLs (UTS, GSR)
Next generation CDN Load Balancer (Project Serena)
Stream manipulation and manifest delivery scaling (DMS 1.0 to SMS)
ISP/CDN QoS data sharing (Blueheeler)
Geo resolution for accurate targeting (VPN detection, partner integrations)
Playback security features (DRM key vending, Token authorization)
Additional Areas Covered
Product documentation portal owner
Owner for Playback wide OE goals and mechanics
Product owner for tenants using PV Playback (Music, Freevee, Kids, MGM+)

Cathy VanAntwerp
cathyva@
TPM
Senior Technical Program Manager: Playback Architecture and Scaling
Owner of cross-org initiative to ensure playback services can functionally withstand peak concurrent streams and extend for future streaming use cases
[CPR/WPR], [Mechanics (MBR/QPR/Roadmap)], [Technical Design], [Cross Team Alignment]
Overall TPM lead for the Andromeda Playback Track: re-architecture, NAWS migration, services decoupling, and world-wide deployment.
Scaling and Resilience: Measuring and increasing PV’s play start services resilience to peak traffic increases and other common failure modes
Resilience engineering - defining and testing playback services resilience strategies for regional failovers and mitigations
Scale Readiness - Ensuring Playback systems are scale ready for Live/VOD launch
Defining strategy for technical approach and migration of Playback 1.0 to Playback 2.0 for VOD and Linear
TPM lead across Playback teams for SWM

Derya Toka
deryat@
PMT
Currently on Leave of Absence, see byde@ for coverage
Sr. Product Manager Tech, Core Playback Capabilities
Product manager for the core services and capabilities that power play starts
Play Starts Strategy and Roadmap: Delivering features that integrate along the play start path, managing device capabilities for vending correct streaming urls, and integrations with entitlement policy store to ensure right resources are vended
Owning the Playback Services 3YP, OP, QPR, MBR, and relevant play start KPIs
Playback Roadmap Mechanics
Core capabilities managed are:
Multi-tiering: Playback Tiering Capabilities to support subscriptions including concurrency rules
Pay-per-view: Foundational work to allow purchasing and playback of live events
Bookmarking: Ability to track customer watch history and improve continued watching

Fernando Flores
redondf@
TPM
Senior Technical Program Manager: Playback Capabilities 
[CPR/WPR], [Technical Design], [Cross Team Alignment]
End-to-end program manager for complex core playback capabilities:
Unified Video Player program which aims to migrate all PV video placements to the latest PV Player available 
Project Cortado (TTFF improvements via new Heuristics, Caching and Play-listing APIs)
VCID to PID Migration - decoupling playback identifiers from catalog.
Multi-tenancy prioritization (Freevee, Music, Kids, MGM+)
Operational Excellence:
Playback wide OE Goals Owner 
Define tooling/infrastructure

Video Analytics, Video Players, and Optimization
The Prime Video Delivery Tech, Optimization and Analytics team manage the close loop flywheel of delivering bits on wire via CDNs and ISPs globally, the player heuristics and algorithm that optimize the video playback and the big data telemetry and analytics systems that form the single source of truth of playback experience for operational and business use cases. The team also manages the programmatic focus of improving Quality of experience of Prime Video customers. 
The Playback Experience team is responsible for the last-mile experience of our customers. We enable 9k+ devices across 6 different platforms. On each device, we provide the ability to play Prime Video, in the best quality, and with innovative customer facing features. We control the look and feel of how our customers interact with Prime Video content. We also maintain the high bar of the video feeds delivered to our customers by using ML based tools (VQA) and building end-to-end testing tools (PTS).

Nandini Iyer
nandiyer@
Sr. Mgr Product
Video Player, Analytics, Optimization  PMT leader
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI] 
Product Strategy and Roadmap for Playback Analytics, QoE metrics, Data Insights and Data Quality
Playback Optimization Strategy and Roadmap
Product Strategy and Roadmap for Players (iOs, Android, XP, Web, LRP)

Nathan Saunders
saundna@
PMT
Principal Product Manager Tech: Mobile Playback Experience
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI]
Product Strategy and Roadmap for iOS and Android 
Product Strategy and Roadmap for Mobile Experience
Product Roadmap for 1P Living Room devices (AppleTV,FireTV)
Mobile Playback Experience interactive Player control features (X-Ray)
Player Decoupling Strategy and Roadmap
Artificial Intelligence Super Resolution (AISR), client upscaling of video resolution, under low bandwidth conditions, to provide high-res video 
Short Form Video 
Autoplay
PV Playback Voice of the Customer Ambassador and Mechanisms

Asher Wittenberg
ahw@
PMT
Principal Product Manager Tech, Playback Analytics & Playback Optimization 
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI]
Data Collection (Playback Telemetry)
Playback QoE Performance Metrics & Ad QoE Measurement
Playback Feature Analytics
Playback QoE Anomaly Detection (Auror)
Playback Analytics Decoupling
Playback Analytics Tools (Sauron)
Automated Data Quality Issue Detection, Prioritization, & Mitigation
Playback Data Privacy & Security
Playback Datasets, Integrations, & APIs
Last Mile Issue Detection, Mitigation, & Customer Messaging (Sonar)
Next-gen Continuous Optimization for Video Heuristics 
Playback Heuristic Treatment Generation, Promotion Service, & Player Simulator
Content & Context Aware Heuristics Optimization
Playback Optimization Algorithms & Science Initiatives
Playback Intake Request Mechanism Owner


Mukesh Sinha
musinha@
TPM
Senior Technical Program Manager
[CPR/MBR/Roadmap], [Launch Readiness/Risk Reviews], [Cross Team Alignment], [Operations Excellence], [Engineering Excellence], [KPI]
Android Player First Party (1P) and Third Party (3P) Program Leader 
Prime Video Player STO on new devices launch from lab126 (Fire SMP, FireTV Edition,Fire Tablets,Alexa Echoshow, Vega MM, Vega TV)
Vega XP Player on Multimodal, TV and Automotive Program Leader
1P, Vega Devices on-boarding, certification and automation
1P  device mechanisms owner including CPR, MBR, QOR
Tech approaches, Execution strategy, HW launches and SW roll out ownership, and Operational deep dives
Stakeholder and Escalation Management (Devices,FireOS,Vega,3P)
FilmMaker Mode on 1P
Regulatory and Compliance Features

Rakhil Kalarithottathil
rkalari@
TPM
Senior Technical Program Manager: Data Quality, Collection and Integrations
[CPR/WPR], [Mechanics (MBR/QBR/Roadmap)], [Technical Design], [Cross Team Alignment]
Data Quality Program to ensure the playback data meets specific business requirements and reflects a true representation of the customer playback experience 
Data Certification including automated testing for Android, iOS, XP player for all streaming technology
Player integration with Playback Test Solution Teams' Aviator tool/framework for data certification
Playback Analytics STO for playback data collection and its specification updates
Auror - Playback QoE anomaly detection program management
Sonar - Last mile playback detection and mitigation program manager
Aastha Niraula

aasthn@
TPM
Technical Program Manager
[CPR/WPR], [Mechanics (MBR/QBR/Roadmap)], [Technical Design], [Cross Team Alignment]
Player Intake Mechanism owner
Player Decoupling Program including UI decoupling ( includes decoupling APis for in-playback features)
Player MBR/CPR owner 
Player Mechanism owner for MTA/subtitles 

Mukul
mukuc@
PMT
Product Manager Tech: Living Room Experience
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI]
Product Strategy and Roadmap for Playback Living Room Experience
Product Roadmap for XP and Web Video Player
Fresh Start Lead 
AppleTV UX Redesign
Vega/VegaTV
Quattro Requirements and Roadmap
FilmMaker Mode 
Multi-Verse and Multi-Players
Living Room Client Liaison
Partner Engineering Liason
Living Room Application Liason
X-Ray Living Room Experience
Next up: VoD, Live and Linear (LR)
Linear CX player 

Quality of Experience (QOE) and Picture Quality
At Prime Video, we serve Live, Linear, and VOD streaming through multiple 1p and 3p channels to customers world-wide on a multitude of devices using various internet providers/infrastructure. Historically, Prime Video has measured the success of our customer quality of experience by a set of KPI’s based on calculations using sums & averages at the stream level implying all streams impact the customer equally. This program (Farpoint) will challenge that assumption by zooming in on a set of measures that distill the impact to a combination of customer, location, ISP, and device related experience to better address the bulk of customers that we can further delight.  Unaddressed, this cohort of customers can lead to loss of trust in Prime Video, drive customers to stop engaging with Prime Video (using other streaming services), and create reputation for Prime Video as unreliable in those cohorts. Our goal is to lead the industry with the best streaming experience worldwide and continue to surprise and delight our customers.

Our team also owns the Page 0 metrics for Prime Video Playback that are used to regularly measure the progress toward the yearly goals.  To drive this, we focus on these  key areas: 1) Metric alignment with PV OP1/OP2 goals by working with DVPI to ensure the correct metrics are surfaced and defined appropriately; 2) Regular checkins (CPR starting Jan-01-2023) to review findings, improvements, and regressions in quality for our customers.  3) Identification of blind spots and addition of metrics to continuously monitor newly identified issues; 4) Working with Player teams, delivery tech, encoding, and any other pertinent team to improve QOE and PQ for Prime Video customers.

The Team

Mikal Christian
mikalc@
SDM
Software Development Manager 
[MBR], required in [CPR/WBR], [KPI], [Technical Design], [Cross Team Alignment], [Page 0 Metrics]
Program lead for QOE in Live, Linear, VOD, and Ads (Program Farpoint)
Driving QOE and PQ improvements with player teams, delivery tech, and playback optimization. 
QOE & Picture Quality Roadmap and Goal setting for Live, Linear, and VOD 
Overall lead for Playback Page 0 metric for QPR and OP document
Do you agree with the list of Success Metrics above?  If not, please contact me.
Continuous improvement on Picture Quality measures
Existing Measures: PHD, PUHD 
Measures to be added: Framerate, bitrate, dropped frames, AV sync, Perceptual metrics (VMAF and/or P.1204) 
Driving these PQ measures and the customer based QOE metrics with DVPI (via ahw@) 

Abinathab Bennet
abinatb@
TPM
Sr. Technical Program Manager 
[MBR], required in [CPR/WBR], [KPI], [Technical Design], [Cross Team Alignment] 
End-to-End program manager for Live Events QoE, Ads QoE and Picture Quality
Lead the CPR for Live QoE, Ads QoE and Picture Quality programs.
Picture Quality Program lead for Live, Linear, and VOD
Goal setting and tracking for Live QoE, Ads QoE and Picture Quality metrics.
QoE and Picture Quality benchmarking(1P and 3P)


Live & Linear Technical Programs
This team leads the execution of new Live Capabilities, including Picture Quality (e.g., HDR, P8) and New Features (e.g., Multi-view). This team will ensure new capabilities are successfully launched and operationally ready for select Tier 1 events, including TNF.  

The Team

Navinan Rajeswaran
navinanr@
TPM
Head of Technical programs for Live & Linear
PV alignment for priority, intake and resourcing
Execution roadmap for Live & Linear
Mechanisms for delivery, quality assurance, launches, operations and reporting
Escalation point of contact for Live & Linear
Tech approaches, Execution Strategy, Tech and Operational Deep Dives
Relationship management for Live and Linear delivery (PV, Amazon, Vendors)
Sye & Ads Integration for Live
Key docs: TNF overall, Live & Linear execution roadmap
Key projects: TNF lead, Audio Lab
Key partners: PV Sports, PV UX, Partner Engineering, AWS


Victoria Koshurina
vct@
TPM
Live TPM lead
CPR lead for Live Capabilities (Live, Sye and Ad Tech)
Live/Linear Issue management & Reporting
Live/Linear execution roadmap & planning
Live/Linear Video Quality & Encoding
New capability and feature rollout for Live/Linear
Tech approaches, Operational Readiness docs, Launch Plans
Live Geography expansion
Partner management
Business docs: MBR, QPR
Key projects:  HDR Expansion, New Capabilities including Multi-View and ATMOS
Key partners: Elemental, PV Sports, Sports Production, PV UX, Partner Engineering, Fire TV
Key Vendors: Dolby, NBC, Switch, IMG, EDM

Ted Trombley
tedtromb@
TPM
Linear TPM lead
Key Contributor for Linear CPR (new capabilities
Linear Issue management & Reporting
Linear execution roadmap & planning
Linear Video Quality & Encoding
New capability and feature rollout for Linear
Tech approaches, Operational Readiness docs, Launch Plans
Linear Geography expansion
Partner management
Business docs: MBR, QPR
Key projects: AHA, Location Based Viewing, Blackouts, Channels Expansion
Key partners: Elemental, PV Sports, PV Channels, PV UX
Key Vendors: Zixi



First, find the quotes from the document that are most relevant to answering the question, and then print them in numbered order. Quotes should be relatively short.

If there are no relevant quotes, write "No relevant quotes" instead.

Then, answer the question, starting with "Answer:". Do not include or reference quoted content verbatim in the answer. Don't say "According to Quote [1]" when answering. Instead make references to quotes relevant to each section of the answer solely by adding their bracketed numbers at the end of relevant sentences.

Thus, the format of your overall response should look like what's shown between the  tags. Make sure to follow the formatting and spacing exactly.


Relevant quotes:
[1] "Ted Trombley
tedtromb@
TPM
Linear TPM lead
Key Contributor for Linear CPR (new capabilities
Linear Issue management & Reporting
Linear execution roadmap & planning
Linear Video Quality & Encoding
New capability and feature rollout for Linear
Tech approaches, Operational Readiness docs, Launch Plans
Linear Geography expansion
Partner management
Business docs: MBR, QPR
Key projects: AHA, Location Based Viewing, Blackouts, Channels Expansion
Key partners: Elemental, PV Sports, PV Channels, PV UX
Key Vendors: Zixi"
[2] 

Answer: 
Ted Trombley is the TPM Lead for Linear and his responsibilities are as follows
Key Contributor for Linear CPR (new capabilities
Linear Issue management & Reporting
Linear execution roadmap & planning
Linear Video Quality & Encoding
New capability and feature rollout for Linear
Tech approaches, Operational Readiness docs, Launch Plans
Linear Geography expansion
Partner management
Business docs: MBR, QPR
Key projects: AHA, Location Based Viewing, Blackouts, Channels Expansion
Key partners: Elemental, PV Sports, PV Channels, PV UX
Key Vendors: Zixi


Here is the first question: In bullet points and simple terms, who is the leader of Playback experience tech product and program management?

If the question cannot be answered by the document, say so.

Answer the question immediately without preamble.


"""

for answer in stream_answer(bedrock_rt, model_id, claude_config, prompt):
    print(answer,end="")

#### <span style="color:#3498db">Optional: Put Words in Claude's Mouth</span>

**Why prefill Claude's response?**

Prefilling Claude's response offers several key benefits:
- **Increased steerability**: By providing some initial text for Claude to continue from, you can steer Claude's response in a desired direction. This is particularly useful when you want Claude to focus on a specific topic, generate a particular type of content, or act a certain way.

- **Control output format**: Prefilling allows you to specify the exact format you want Claude to use for its output. This is especially handy when working with structured data formats like JSON or XML. For more details on this, see our guide on controlling output format.

- **Maintain character consistency**: In role-play scenarios, prefilling Claude's response can help Claude stay in character throughout a long conversation. By consistently reminding Claude of its role in the Assistant message, you can better ensure that Claude maintains the desired persona. Check out keep Claude stay in character for more details.


In [ ]:

def generate_answer(bedrock_rt:boto3.client, prompt:str, model_id:str, claude_config:dict,system_prompt:str)->str:
    

    message={'messages': [{"role": "user", "content": prompt},
                          {"role": "assistant", "content": "The answer is"}]}
    
    body={**message,**claude_config, "system": system_prompt}
    
    response = bedrock_rt.invoke_model(modelId=model_id, body=json.dumps(body))
    response = json.loads(response['body'].read().decode('utf-8'))
    return response['content'][0]['text']

In [27]:
system_prompt="""

Human: I'm going to give you a document. Then I'm going to ask you a question about it. I'd like you to first write down exact quotes of parts of the document that would help answer the question, and then I'd like you to answer the question using facts from the quoted content. Here is the document:

Playback Experience Tech - Product and Program Team 

Team Overview
Playback brings the technologies, tools, and operations to stream Live, Linear, and VOD content to all devices globally. Our infrastructure brings flawless, high-quality streaming while using the fewest bits possible. We aim to deliver a better experience than broadcast. We innovate to efficiently manage our global scale across CDNs, ISPs, every sort of home network, and 10K+ device models. Video playback includes acquiring the video (and audio) content from partners, Encoding (compression) of the video, policy enforcement to meet locale and studio requirements, measuring the quality of the experience (QoE), vending of all playback resources, managing the storage of the content, delivery of the video fragments over CDN / ISP networks to the customer's device and the playing back of the video on the customer's device including UI control of the video player. Playback also manages our operational team responsible for monitoring and mitigating our video delivery solution, including VOD, Live Events, and Linear channels. We also constructed the Analytics system that measures our video playback, including an AI-based monitoring and alerting system to identify issues in real-time. We can then root causes and swiftly mitigate these on a global scale. The data generated by the Playback system is the source of truth for video consumption and usage. We share the data in a privacy-safe manner across all stakeholders. In addition, our infrastructure optimizes video delivery across the network in real-time so viewer always get the best possible quality that their network and device is capable of supporting. Finally, we provide video playback to our tenants, such as Freevee and Music, bringing economies of scale and innovation to Amazon video properties.

We split our team into the following product and program clusters:

Encoding and Picture Quality (PQ)
Live Streaming - Live Events and Linear
Playback Services and Video Delivery
Video Player, Playback Experience, Video Analytics, Optimization
Live and Linear Program Management
Quality of the Experience (QoE) and Picture Quality (PQ) Program Management
Recent Updates
See our new Documentation Portal!

https://w.amazon.com/bin/download/Amazon_Video/PV_Playback/Product_and_Program/New_Home/WebHome/docs.png

Mission
Prime Video Playback technologies are the benchmark for video streaming, providing the best viewing experience for our customers globally. Amazon's best-in-class library (VOD and Live) will always be the preferred choice for customer entertainment anywhere and on any device. Our applications and services are secure, protecting content (Amazon and partners), customer information, and our business. In the future, our solution will allow us to support >500MM active customers worldwide in the most cost-effective way possible while enabling multi-tenants to promote their video business expansion.

Our mission is to build and operate the most innovative video streaming technology that provides the best streaming for VOD and Live across all devices globally. We measure our success by delivering the earth's most customer-centric streaming globally. We built a multi-tenant stack used and extended by other organizations in Amazon. We secure our digital content to maintain trust with our partners.

Our Tenets (Unless You Know Better...)
We willingly adopt Audio/Video (A/V) streaming standards and implement them at scale. We invent new A/V tech without constraints on behalf of our customers, influencing original equipment manufacturers (OEM), content delivery networks (CDN), internet service providers (ISP), industry partners, and standards bodies to adopt our proposals and innovate with us. We do not merely adopt emerging tech. We strive to invent them.
We operate our services by continually improving 1) reliability and availability, 2) video experience and picture quality, and 3) new customer-facing capabilities. When we have to make tradeoffs, we prioritize in the order listed.
We design our services with multi-tenancy and decoupling in mind. Our stakeholders shouldn't rely on us to deliver for their customers. When we slow them down, we'll take a step back and assess how to enable them to move fast and increase velocity.
Delivering video at scale reliably is a complex problem involving complicated tech and expertise. We are biased towards developing and operating these technologies in-house and fall back to third-party (3P) technology when we see an opportunity to accelerate customer benefit.
We will continuously lower the cost of bits-on-the-wire while ensuring we don't compromise the perceived customer viewing experience.
We launch new products and services with scalable, automated processes. Any exception is a highly considered decision with a swift plan to close the gaps. We will not launch a new feature directly on high-profile content. We will instead iteratively deploy on low-impact and low-risk content. When we have high confidence, we will roll it out for high-profile content.

Team Overview
Our team is responsible for the sound development of the playback solution. The team assesses, defines, prioritizes, and supports the development of solutions for the video playback organization. We prioritizes our roadmap based on our customer obsession - across the video playback organization. The team defines and captures the strategic vision, establishes the roadmap, defines the requirements, and works closely with engineering teams to execute on our vision. The groups encompass the entire video pipeline, including Video Players (Android, Web, iOS, Living Room), Encoding, Live Playback, Video Delivery, Playback Insights, Content Access, Playback Services, and Playback Experience. We drive Product Mechanics, focusing on a well-understood cadence, transparency, accountability, and customer value. We write 3-year Vision documents, PRFAQs, BRDs, Product Goals and metrics, and requirements documents outlining the customer experience. We jointly work with engineering teams to define our technical approach and workback plans. We are customer-obsessed and partner with all stakeholders to ensure we deliver the best experience for our customers. We synchronize with the industry to influence their roadmaps and technology decisions to align with Prime Video's vision. We determine long-term leapfrogging ideas for the future of digital video consumption, focusing the team on the execution of the next-generation playback technologies and experiences that delight our customers.

Success Metrics
Customer Experience & Satisfaction
We measure our performance holistically using several metrics for each critical aspect of our business. Our solutions aim to improve key business metrics such as 1) Hours Streamed (Absolute and CAGR), 2) Peak Concurrent Plays (Absolute and CAGR), or 3) Number of Monthly Active Users (MAU). We group Playback critical metrics into the following categories: 1) measure our ability to scale, 2) the Quality of the Experience (QoE) rendered to our viewer, 3) the perceptual Picture Quality (PQ) of our streaming, 4) the customer satisfaction driven by our video Playback capabilities, and 5) our Cost and Operational Efficiency of running the video infrastructure.

We measure our Scale and Availability with Peak Concurrent Plays (PCP) sessions (in Million/M), # Video Streams (in Billion/BN), # Viewing Hours Streamed without issues (in Billion/BN), # Countries, # Linear Stations, # Live Events, # Live Events on Linear (LEL), # VOD Titles (in MM titles), # of Broadcast Hours for all Live Events, Total Bytes Delivered (in Petabyte/PB) and % of Traffic delivered with CloudFront
We assess our customer Quality of Experience (QoE) with % Time Spent Buffering, Time-to-First-Frame (TTFF), Zero Error Rate (ZER), Zero Buffer Rate (ZBR), Exit Before Video Start (EBVS), and Buffering Abandoned Plays (BAP). We strike a balance to optimize for these metrics, aiming to drive a "better than broadcast" experience given constraints such as available bandwidth, cap limitation, and cost
We assess our customer Picture Quality (PQ) by leveraging the % of HD and UHD fragments for suitable sessions. We optimize for High Dynamic Range (HDR) and Dolby Vision coverage. We also use subjective testing and golden eyes to assess our quality. We introduce new measurements with Video Multi-method Assessment Fusion (VMAF at P90, Amazon Quality Measurement, ITU-T REC P.1204) to improve our quantification in this area. These PQ metrics use mathematical models to compute the video quality automatically
We assess Customer Satisfaction when we launch new capabilities and value brought by our video experience - usability testing, NPS, or other proxies such as social metrics or press clipping. We evaluate customer engagement with playback capabilities - Reviews & feedback, Customer sentiment score, app store ratings for video playback
Finally, we obsess about our Cost and Operation Efficiency with KPIs such as Playback Cost Per 1000 Streams (CPS), Encode cost Per Hour (EPH), CDN cost, and per Linear station cost. We also measure our operational and monitoring efficiency by leveraging KPIs such as Operators per Live Events or Linear stations.
Finally, we aim to constantly improve the speed at which we deliver new capabilities to our customers. We strive for faster roadmap velocity and value brought to our customers and stakeholders. The product & program team must increase predictability with a sustained cadence understood by all. We reduce blockers reported and lower scope-cuts/scope-creep to meet deadlines and expectations. We support engineering in reducing counts of defects reported during delivery and better handling of quality levels. Finally, our team aims to reduce our churn by making our work enjoyable with a purpose for both the team and our stakeholders - continuously working on our connection scores and bringing the maximum value to our customers.

Product & Program Velocity
We aim to constantly improve the speed at which we deliver new capabilities to our customers. We strive for faster roadmap velocity and value brought to our customers and stakeholders. The product & program team must increase predictability with a sustained cadence understood by all. We reduce blockers reported and lower scope-cuts/scope-creep to meet deadlines and expectations. We support engineering in reducing counts of defects reported during delivery and better handling of quality levels. Finally, our team aims to reduce our churn by making our work enjoyable with a purpose for both the team and our stakeholders - continuously working on our connection scores and bringing the maximum value to our customers.

Leadership
Olivier Wellmann
wellmo@
Director
Director of Product and Program Management 
[3-year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [CPR/WBR], [KPI]
Video Playback Product Strategy across Amazon Business Cadence (3YP, OP, QPR)
Product Roadmap and Execution for Video Playback
Technical Product Management Leader
Technical Program Management Leader
Product Goals for Playback
Recruiting, coaching, and mentoring
Product and Program Escalation

Martha Cesari
cehmarth@
EA
Executive Assistant 
Manages Olivier's calendar

Video Innovation
Innovation focuses on rising the bar in Camera-to-Cornea customer experience though industry and stakeholder collaboration, whole product perspective, and get patents. Prime Video works with many external and internal partners from content producers to post houses, validation and analysis tools, encoding and packaging tools, CDNs, and consumer electronics companies. As new technologies and features constantly emerge, we need to focus on those that will provide customer value, and work with partners to bring them to market quickly, efficiently, and with high quality. Much is done bilaterally, delivering Prime Video first technologies like HDR10+. We also build upon many industry standards, and Prime Video contributes to standards that raise the bar for customers, like Filmmaker Mode.
While Prime Video is made by a huge number of teams and individuals all working on aspects of the service, Our customers receive Prime Video as a whole product. We innovate looking at the big picture for cross-team bar-raising efforts. Examples include recommending cameras and settings for Amazon Studios, delivering more consistent audio loudness, and providing customers optimum picture modes.

Ben Waggoneer
benwagg@
TPM
Principal Video Specialist 
Video Technology Leader and Subject Matter Expert
Prime Video Audio/Video Evangelist
Emerging Technology evaluation and proofs of concept,
player control over picture modes, new media technologies, VR, 3D, multi primary video
Quality Innovation, next-gen video (AV1, VVC AV2), 8K and UHFR (100 and 120 fps) initiatives, ambient light adaptation, dynamic metadata, heuristic development, lens-to-screen whole product quality and accuracy.
Audio Quality Innovation, including next gen spatial audio (MPEG-H, AC4, IAMF), loudness control, dialog clarity, and other audio enhancements
Streaming Product Architecture Improvements
Video Industry Partnerships, including UHD Alliance (UHDA), UHD Forum (UHDF), 8K Association, and SMPTE
Bidirectional innovation partnerships with studios and CE companies
Technology Patenting


Live Event & Linear, Live Operations & Monitoring, VOD Encoding & AV Quality
Live Events & Linear: The Live Event and Linear Product Teams owns the live events and linear product offerings for Playback. Our mission is to build and operate the world’s most innovative live video streaming technology stack, providing a customer-centric live streaming experience. Our desire is to set the bar for the industry and position PV as the top streaming service in the world by being significantly better than broadcast TV in terms of availability, quality, reliability, and scale for live streaming.
Live Operations & Monitoring: The Live Operations and Live Monitoring Product teams owns the operation and end-to-end monitoring of all Live events and Linear stations on Prime Video. The program ensures we can rapidly onboard and operate events and linear stations at scale, including automation (all processes from pre-event to post-event), best-in-class 24/7 monitoring (finding defects before customers tell us), and actively monitor and triage Live event issues to mitigate core playback issues in <3 Minutes. We measure success by the number of events and Linear stations we delivered without issues. We continuously drive operational efficiency to improve our ability to swiftly launch events and stations cost-effectively.
VOD Encoding & AV Quality: The VOD Encoding Product Team owns the generation of all VOD media (card insertion, ad insertion, and main title variants) and is responsible for all VOD picture and audio quality, content compression (and the associated globally distributed storage) using various encoding algorithms and packaging of the content to support the audio/video capabilities for 10K+ customer devices. This team is responsible for building the pipelines for the analysis, preprocessing, encoding, packaging, and encryption of video and audio streams, as well as delivering the highest quality consumer content by building sufficient guardrails in the systems and services. The Encoding team mission is to create the highest quality video, audio, captions and images possible for Prime Video customers worldwide.

Scott Mackenzie
scmacken@
Sr. Mgr Product
Playback Product Lead for Live Events, Linear, VOD Encoding, Video Quality Analysis and Live Operations Technology
Live Events Playback Product Leader
Linear Playback Product Leader
VOD Encoding Playback Product and Program Leader
Video Quality Analysis Playback Product Leader
Live Operations Technology Product Leader
[3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Playback Product Strategy (3YP, OP, QPR, MBR)
Playback Product Roadmap and Execution
Playback Product and Program Recruiting
Playback Product and Program Escalation
Playback Product and Program Owner
Heddy Kuhl

kuhheddy@
PMT
Senior Product Manager Tech for Live Events
Playback Live Event Product Leader
Playback Live Event [3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Live Sports Org Playback Partner
Playback Live Event Product Owner & Strategy
Playback Live Event Product Roadmap and Execution
Playback Live Event Capabilities (i.e. time-shifting, bitrate profiles, CX features)
Playback Live Signal Ingestion of Live Events and Concerts
Playback Live Event Onboarding, including transcoding, orchestration, content origin, and live monitoring
Playback Live Infrastructure, encoding, packaging and origin
Playback Live Encoding, a/v codecs and bitrate profiles
Partner Management for AWS Elemental Roadmap


Oleg Tsaregorodstev
oletsare@
PMT
Principal Product Manager Tech for Linear TV
Playback Linear Product Leader
[3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Playback Linear Product Strategy (3YP, OP, QPR, MBR)
Playback Linear Product Roadmap and Execution
Playback Linear Product Owner
Playback Linear Signal Ingestion
Playback Linear Infrastructure, including encoding, packaging and origin
Free Ad-supported Streaming TV (FAST)
Sports rights Ad-Hoc Aggregators (AHA)
Live Events on Linear
Linear Capabilities (e.g., time-shifting, multi-track audio)

Max Denton
maxdent@
PMT
Principal Product Manager Tech for VOD Encoding
Playback VOD Encoding Product Leader
[3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Playback VOD Encoding Product Strategy (3YP, OP, QPR, MBR)
Playback VOD Encoding Roadmap and Execution
[CPR/WPR], [Mechanics (MBR/QPR/Roadmap)], [Technical Design], [Cross Team Alignment]
Gen 4 Next Generation Encoding Pipeline Product Leader
Card Stitching Program Leader (End to End)
Encoding Latency On-Time Publishing Product Leader
Encoding Infrastructure Cost Out Product Leader
VOD Ads scaling
VOD AV1 roll out

Andrew Whittaker
awhittak@
TPM
Playback Sr. Technical Program Manager (TPM)
PV single threaded TPM leader for Card Stitching feature
VOD Encoding Card Stitching (CS) Program Leader
Standing in for PV single threaded PMT leader for Card Stitching feature.
TPM for Playback Encoding
[CPR/WPR], [Mechanics (MBR/QPR/Roadmap)], [Technical Design], [Cross Team Alignment]
Gen 4 Next Generation Encoding Pipeline Program
Latency On-Time Publishing Program
Infrastructure Cost Out Program
Ads encoding scaling
AV1 roll out
Team Mentoring

Joel Korpi
korpijk@
PMT
Senior Product Manager Tech for VOD Encoding, Picture Quality and Codecs
Currently on Leave of Absence, see scmacken@ for coverage
Playback VOD Encoding Product Leader
[3 Year Vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], [KPI]
Playback VOD Encoding Picture Quality and Codec Product Strategy (3YP, OP, QPR, MBR)
Playback VOD Encoding Picture Quality, Codec Roadmap and Execution
Playback VOD Codec strategy, including AV1/AV2 (Alliance for Open Media) and Versatile Video Coding (VVC/H.266)
Playback VOD R&D, including Cinema+ and TruColor
Partner Management for Dolby Laboratories

Playback Services and Video Delivery
This product area owns the end-to-end flow from when the customer hits play until their playback session is complete.  We build the integration point between backend services and customer-facing video players to 1) deliver the correct stream entitlement, 2) optimize manifests for the customer and device, and 3) enforce the concurrency limits set by subscription policies. It also covers services that protect content (DRM, Geo Services, VPN abuse) and tracks user sessions to ensure they can resume where they left off on any device. Our video delivery services takes the encoded bits from content origins and delivers the asset to our customer's devices over the internet. We continuously ensure the availability and network capacity to bring millions of concurrent streams to customers reliably across CDNs, ISPs, mobile networks, and home networks. We build with AWS and 3P providers, caching capacity and content as close to the customer as possible. Using real-time data and ML optimization, we determine the most cost-effective network path for a customer session while providing the highest quality stream.

Eric Boyd
byde@
Sr. Mgr Product
Sr. Manager Product, Playback Services and Video Delivery
Product lead for video delivery and play start services that deliver playable streams to customers world-wide
Product Strategy and Roadmap Execution (3YP, OP, MBR, QPR)
CDN edge authorization (Edge tokenization)
Granular Session Routing and optimized CDN URLs (UTS, GSR)
Next generation CDN Load Balancer (Project Serena)
Stream manipulation and manifest delivery scaling (DMS 1.0 to SMS)
ISP/CDN QoS data sharing (Blueheeler)
Geo resolution for accurate targeting (VPN detection, partner integrations)
Playback security features (DRM key vending, Token authorization)
Additional Areas Covered
Product documentation portal owner
Owner for Playback wide OE goals and mechanics
Product owner for tenants using PV Playback (Music, Freevee, Kids, MGM+)

Cathy VanAntwerp
cathyva@
TPM
Senior Technical Program Manager: Playback Architecture and Scaling
Owner of cross-org initiative to ensure playback services can functionally withstand peak concurrent streams and extend for future streaming use cases
[CPR/WPR], [Mechanics (MBR/QPR/Roadmap)], [Technical Design], [Cross Team Alignment]
Overall TPM lead for the Andromeda Playback Track: re-architecture, NAWS migration, services decoupling, and world-wide deployment.
Scaling and Resilience: Measuring and increasing PV’s play start services resilience to peak traffic increases and other common failure modes
Resilience engineering - defining and testing playback services resilience strategies for regional failovers and mitigations
Scale Readiness - Ensuring Playback systems are scale ready for Live/VOD launch
Defining strategy for technical approach and migration of Playback 1.0 to Playback 2.0 for VOD and Linear
TPM lead across Playback teams for SWM

Derya Toka
deryat@
PMT
Currently on Leave of Absence, see byde@ for coverage
Sr. Product Manager Tech, Core Playback Capabilities
Product manager for the core services and capabilities that power play starts
Play Starts Strategy and Roadmap: Delivering features that integrate along the play start path, managing device capabilities for vending correct streaming urls, and integrations with entitlement policy store to ensure right resources are vended
Owning the Playback Services 3YP, OP, QPR, MBR, and relevant play start KPIs
Playback Roadmap Mechanics
Core capabilities managed are:
Multi-tiering: Playback Tiering Capabilities to support subscriptions including concurrency rules
Pay-per-view: Foundational work to allow purchasing and playback of live events
Bookmarking: Ability to track customer watch history and improve continued watching

Fernando Flores
redondf@
TPM
Senior Technical Program Manager: Playback Capabilities 
[CPR/WPR], [Technical Design], [Cross Team Alignment]
End-to-end program manager for complex core playback capabilities:
Unified Video Player program which aims to migrate all PV video placements to the latest PV Player available 
Project Cortado (TTFF improvements via new Heuristics, Caching and Play-listing APIs)
VCID to PID Migration - decoupling playback identifiers from catalog.
Multi-tenancy prioritization (Freevee, Music, Kids, MGM+)
Operational Excellence:
Playback wide OE Goals Owner 
Define tooling/infrastructure

Video Analytics, Video Players, and Optimization
The Prime Video Delivery Tech, Optimization and Analytics team manage the close loop flywheel of delivering bits on wire via CDNs and ISPs globally, the player heuristics and algorithm that optimize the video playback and the big data telemetry and analytics systems that form the single source of truth of playback experience for operational and business use cases. The team also manages the programmatic focus of improving Quality of experience of Prime Video customers. 
The Playback Experience team is responsible for the last-mile experience of our customers. We enable 9k+ devices across 6 different platforms. On each device, we provide the ability to play Prime Video, in the best quality, and with innovative customer facing features. We control the look and feel of how our customers interact with Prime Video content. We also maintain the high bar of the video feeds delivered to our customers by using ML based tools (VQA) and building end-to-end testing tools (PTS).

Nandini Iyer
nandiyer@
Sr. Mgr Product
Video Player, Analytics, Optimization  PMT leader
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI] 
Product Strategy and Roadmap for Playback Analytics, QoE metrics, Data Insights and Data Quality
Playback Optimization Strategy and Roadmap
Product Strategy and Roadmap for Players (iOs, Android, XP, Web, LRP)

Nathan Saunders
saundna@
PMT
Principal Product Manager Tech: Mobile Playback Experience
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI]
Product Strategy and Roadmap for iOS and Android 
Product Strategy and Roadmap for Mobile Experience
Product Roadmap for 1P Living Room devices (AppleTV,FireTV)
Mobile Playback Experience interactive Player control features (X-Ray)
Player Decoupling Strategy and Roadmap
Artificial Intelligence Super Resolution (AISR), client upscaling of video resolution, under low bandwidth conditions, to provide high-res video 
Short Form Video 
Autoplay
PV Playback Voice of the Customer Ambassador and Mechanisms

Asher Wittenberg
ahw@
PMT
Principal Product Manager Tech, Playback Analytics & Playback Optimization 
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI]
Data Collection (Playback Telemetry)
Playback QoE Performance Metrics & Ad QoE Measurement
Playback Feature Analytics
Playback QoE Anomaly Detection (Auror)
Playback Analytics Decoupling
Playback Analytics Tools (Sauron)
Automated Data Quality Issue Detection, Prioritization, & Mitigation
Playback Data Privacy & Security
Playback Datasets, Integrations, & APIs
Last Mile Issue Detection, Mitigation, & Customer Messaging (Sonar)
Next-gen Continuous Optimization for Video Heuristics 
Playback Heuristic Treatment Generation, Promotion Service, & Player Simulator
Content & Context Aware Heuristics Optimization
Playback Optimization Algorithms & Science Initiatives
Playback Intake Request Mechanism Owner


Mukesh Sinha
musinha@
TPM
Senior Technical Program Manager
[CPR/MBR/Roadmap], [Launch Readiness/Risk Reviews], [Cross Team Alignment], [Operations Excellence], [Engineering Excellence], [KPI]
Android Player First Party (1P) and Third Party (3P) Program Leader 
Prime Video Player STO on new devices launch from lab126 (Fire SMP, FireTV Edition,Fire Tablets,Alexa Echoshow, Vega MM, Vega TV)
Vega XP Player on Multimodal, TV and Automotive Program Leader
1P, Vega Devices on-boarding, certification and automation
1P  device mechanisms owner including CPR, MBR, QOR
Tech approaches, Execution strategy, HW launches and SW roll out ownership, and Operational deep dives
Stakeholder and Escalation Management (Devices,FireOS,Vega,3P)
FilmMaker Mode on 1P
Regulatory and Compliance Features

Rakhil Kalarithottathil
rkalari@
TPM
Senior Technical Program Manager: Data Quality, Collection and Integrations
[CPR/WPR], [Mechanics (MBR/QBR/Roadmap)], [Technical Design], [Cross Team Alignment]
Data Quality Program to ensure the playback data meets specific business requirements and reflects a true representation of the customer playback experience 
Data Certification including automated testing for Android, iOS, XP player for all streaming technology
Player integration with Playback Test Solution Teams' Aviator tool/framework for data certification
Playback Analytics STO for playback data collection and its specification updates
Auror - Playback QoE anomaly detection program management
Sonar - Last mile playback detection and mitigation program manager
Aastha Niraula

aasthn@
TPM
Technical Program Manager
[CPR/WPR], [Mechanics (MBR/QBR/Roadmap)], [Technical Design], [Cross Team Alignment]
Player Intake Mechanism owner
Player Decoupling Program including UI decoupling ( includes decoupling APis for in-playback features)
Player MBR/CPR owner 
Player Mechanism owner for MTA/subtitles 

Mukul
mukuc@
PMT
Product Manager Tech: Living Room Experience
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI]
Product Strategy and Roadmap for Playback Living Room Experience
Product Roadmap for XP and Web Video Player
Fresh Start Lead 
AppleTV UX Redesign
Vega/VegaTV
Quattro Requirements and Roadmap
FilmMaker Mode 
Multi-Verse and Multi-Players
Living Room Client Liaison
Partner Engineering Liason
Living Room Application Liason
X-Ray Living Room Experience
Next up: VoD, Live and Linear (LR)
Linear CX player 

Quality of Experience (QOE) and Picture Quality
At Prime Video, we serve Live, Linear, and VOD streaming through multiple 1p and 3p channels to customers world-wide on a multitude of devices using various internet providers/infrastructure. Historically, Prime Video has measured the success of our customer quality of experience by a set of KPI’s based on calculations using sums & averages at the stream level implying all streams impact the customer equally. This program (Farpoint) will challenge that assumption by zooming in on a set of measures that distill the impact to a combination of customer, location, ISP, and device related experience to better address the bulk of customers that we can further delight.  Unaddressed, this cohort of customers can lead to loss of trust in Prime Video, drive customers to stop engaging with Prime Video (using other streaming services), and create reputation for Prime Video as unreliable in those cohorts. Our goal is to lead the industry with the best streaming experience worldwide and continue to surprise and delight our customers.

Our team also owns the Page 0 metrics for Prime Video Playback that are used to regularly measure the progress toward the yearly goals.  To drive this, we focus on these  key areas: 1) Metric alignment with PV OP1/OP2 goals by working with DVPI to ensure the correct metrics are surfaced and defined appropriately; 2) Regular checkins (CPR starting Jan-01-2023) to review findings, improvements, and regressions in quality for our customers.  3) Identification of blind spots and addition of metrics to continuously monitor newly identified issues; 4) Working with Player teams, delivery tech, encoding, and any other pertinent team to improve QOE and PQ for Prime Video customers.

The Team

Mikal Christian
mikalc@
SDM
Software Development Manager 
[MBR], required in [CPR/WBR], [KPI], [Technical Design], [Cross Team Alignment], [Page 0 Metrics]
Program lead for QOE in Live, Linear, VOD, and Ads (Program Farpoint)
Driving QOE and PQ improvements with player teams, delivery tech, and playback optimization. 
QOE & Picture Quality Roadmap and Goal setting for Live, Linear, and VOD 
Overall lead for Playback Page 0 metric for QPR and OP document
Do you agree with the list of Success Metrics above?  If not, please contact me.
Continuous improvement on Picture Quality measures
Existing Measures: PHD, PUHD 
Measures to be added: Framerate, bitrate, dropped frames, AV sync, Perceptual metrics (VMAF and/or P.1204) 
Driving these PQ measures and the customer based QOE metrics with DVPI (via ahw@) 

Abinathab Bennet
abinatb@
TPM
Sr. Technical Program Manager 
[MBR], required in [CPR/WBR], [KPI], [Technical Design], [Cross Team Alignment] 
End-to-End program manager for Live Events QoE, Ads QoE and Picture Quality
Lead the CPR for Live QoE, Ads QoE and Picture Quality programs.
Picture Quality Program lead for Live, Linear, and VOD
Goal setting and tracking for Live QoE, Ads QoE and Picture Quality metrics.
QoE and Picture Quality benchmarking(1P and 3P)


Live & Linear Technical Programs
This team leads the execution of new Live Capabilities, including Picture Quality (e.g., HDR, P8) and New Features (e.g., Multi-view). This team will ensure new capabilities are successfully launched and operationally ready for select Tier 1 events, including TNF.  

The Team

Navinan Rajeswaran
navinanr@
TPM
Head of Technical programs for Live & Linear
PV alignment for priority, intake and resourcing
Execution roadmap for Live & Linear
Mechanisms for delivery, quality assurance, launches, operations and reporting
Escalation point of contact for Live & Linear
Tech approaches, Execution Strategy, Tech and Operational Deep Dives
Relationship management for Live and Linear delivery (PV, Amazon, Vendors)
Sye & Ads Integration for Live
Key docs: TNF overall, Live & Linear execution roadmap
Key projects: TNF lead, Audio Lab
Key partners: PV Sports, PV UX, Partner Engineering, AWS


Victoria Koshurina
vct@
TPM
Live TPM lead
CPR lead for Live Capabilities (Live, Sye and Ad Tech)
Live/Linear Issue management & Reporting
Live/Linear execution roadmap & planning
Live/Linear Video Quality & Encoding
New capability and feature rollout for Live/Linear
Tech approaches, Operational Readiness docs, Launch Plans
Live Geography expansion
Partner management
Business docs: MBR, QPR
Key projects:  HDR Expansion, New Capabilities including Multi-View and ATMOS
Key partners: Elemental, PV Sports, Sports Production, PV UX, Partner Engineering, Fire TV
Key Vendors: Dolby, NBC, Switch, IMG, EDM

Ted Trombley
tedtromb@
TPM
Linear TPM lead
Key Contributor for Linear CPR (new capabilities
Linear Issue management & Reporting
Linear execution roadmap & planning
Linear Video Quality & Encoding
New capability and feature rollout for Linear
Tech approaches, Operational Readiness docs, Launch Plans
Linear Geography expansion
Partner management
Business docs: MBR, QPR
Key projects: AHA, Location Based Viewing, Blackouts, Channels Expansion
Key partners: Elemental, PV Sports, PV Channels, PV UX
Key Vendors: Zixi



First, find the quotes from the document that are most relevant to answering the question, and then print them in numbered order. Quotes should be relatively short.

If there are no relevant quotes, write "No relevant quotes" instead.

Then, answer the question, starting with "Answer:". Do not include or reference quoted content verbatim in the answer. Don't say "According to Quote [1]" when answering. Instead make references to quotes relevant to each section of the answer solely by adding their bracketed numbers at the end of relevant sentences.

Thus, the format of your overall response should look like what's shown between the  tags. Make sure to follow the formatting and spacing exactly.


Relevant quotes:
[1] "Ted Trombley
tedtromb@
TPM
Linear TPM lead
Key Contributor for Linear CPR (new capabilities
Linear Issue management & Reporting
Linear execution roadmap & planning
Linear Video Quality & Encoding
New capability and feature rollout for Linear
Tech approaches, Operational Readiness docs, Launch Plans
Linear Geography expansion
Partner management
Business docs: MBR, QPR
Key projects: AHA, Location Based Viewing, Blackouts, Channels Expansion
Key partners: Elemental, PV Sports, PV Channels, PV UX
Key Vendors: Zixi"
[2] 

Answer: 
Ted Trombley is the TPM Lead for Linear and his responsibilities are as follows
Key Contributor for Linear CPR (new capabilities
Linear Issue management & Reporting
Linear execution roadmap & planning
Linear Video Quality & Encoding
New capability and feature rollout for Linear
Tech approaches, Operational Readiness docs, Launch Plans
Linear Geography expansion
Partner management
Business docs: MBR, QPR
Key projects: AHA, Location Based Viewing, Blackouts, Channels Expansion
Key partners: Elemental, PV Sports, PV Channels, PV UX
Key Vendors: Zixi


"""

prompt="""
Here is the first question: In bullet points and simple terms, who is the leader of Playback experience tech product and program management?

If the question cannot be answered by the document, say so.

Answer the question immediately without preamble.
"""
response=generate_answer(bedrock_rt, prompt, model_id, claude_config,system_prompt)
print(colored(response, "green"))

:

- Olivier Wellmann (wellmo@) is the Director of Product and Program Management for Playback Experience Tech.

His responsibilities include:

- 3-year vision, OP (Operating Plan), PRFAQ (Frequently Asked Questions), BRD (Business Requirements Document), QPR (Quarterly Program Review), MBR (Monthly Business Review), CPR/WBR (Commitment/Work-back Review), KPI (Key Performance Indicators)
- Video Playback Product Strategy across Amazon Business Cadence
- Product Roadmap and Execution for Video Playback
- Technical Product Management Leader
- Technical Program Management Leader
- Product Goals for Playback
- Recruiting, coaching, and mentoring
- Product and Program Escalation


In [28]:
prompt="Who owns Live and Linear Technical Programs?"
response=generate_answer(bedrock_rt, prompt, model_id, claude_config,system_prompt)
print(colored(response, "green"))

 provided in this quote:

[1] "Navinan Rajeswaran
navinanr@
TPM
Head of Technical programs for Live & Linear
PV alignment for priority, intake and resourcing
Execution roadmap for Live & Linear
Mechanisms for delivery, quality assurance, launches, operations and reporting
Escalation point of contact for Live & Linear
Tech approaches, Execution Strategy, Tech and Operational Deep Dives
Relationship management for Live and Linear delivery (PV, Amazon, Vendors)
Sye & Ads Integration for Live
Key docs: TNF overall, Live & Linear execution roadmap
Key projects: TNF lead, Audio Lab
Key partners: PV Sports, PV UX, Partner Engineering, AWS"

Navinan Rajeswaran is the Head of Technical Programs for Live & Linear. His responsibilities include alignment on priorities and resourcing, execution roadmap, delivery mechanisms, escalation point of contact, technical approaches and strategy, relationship management with partners and vendors for Live and Linear delivery. [1]


In [29]:
prompt="What are the team's tenets?"
response=generate_answer(bedrock_rt, prompt, model_id, claude_config,system_prompt)
print(colored(response, "green"))

 based on the following quotes from the document:

[3] "Our Tenets (Unless You Know Better...)
We willingly adopt Audio/Video (A/V) streaming standards and implement them at scale. We invent new A/V tech without constraints on behalf of our customers, influencing original equipment manufacturers (OEM), content delivery networks (CDN), internet service providers (ISP), industry partners, and standards bodies to adopt our proposals and innovate with us. We do not merely adopt emerging tech. We strive to invent them.
We operate our services by continually improving 1) reliability and availability, 2) video experience and picture quality, and 3) new customer-facing capabilities. When we have to make tradeoffs, we prioritize in the order listed.
We design our services with multi-tenancy and decoupling in mind. Our stakeholders shouldn't rely on us to deliver for their customers. When we slow them down, we'll take a step back and assess how to enable them to move fast and increase velocity.


In [30]:
prompt="What does the QoE team do?"
response=generate_answer(bedrock_rt, prompt, model_id, claude_config,system_prompt)
print(colored(response, "green"))

 based on these relevant quotes:

[1] "At Prime Video, we serve Live, Linear, and VOD streaming through multiple 1p and 3p channels to customers world-wide on a multitude of devices using various internet providers/infrastructure. Historically, Prime Video has measured the success of our customer quality of experience by a set of KPI's based on calculations using sums & averages at the stream level implying all streams impact the customer equally. This program (Farpoint) will challenge that assumption by zooming in on a set of measures that distill the impact to a combination of customer, location, ISP, and device related experience to better address the bulk of customers that we can further delight."

[2] "Our team also owns the Page 0 metrics for Prime Video Playback that are used to regularly measure the progress toward the yearly goals. To drive this, we focus on these key areas: 1) Metric alignment with PV OP1/OP2 goals by working with DVPI to ensure the correct metrics are surfac

In [31]:
prompt="Who should I reach out for analytics in the team?"
response=generate_answer(bedrock_rt, prompt, model_id, claude_config,system_prompt)
print(colored(response, "green"))

 based on the following relevant quotes:

[1] "Asher Wittenberg
ahw@
PMT
Principal Product Manager Tech, Playback Analytics & Playback Optimization 
[3 year vision], [OP], [PRFAQ], [BRD], [QPR], [MBR], required in [CPR/WBR], [KPI]
Data Collection (Playback Telemetry)
Playback QoE Performance Metrics & Ad QoE Measurement
Playback Feature Analytics
Playback QoE Anomaly Detection (Auror)
Playback Analytics Decoupling
Playback Analytics Tools (Sauron)
Automated Data Quality Issue Detection, Prioritization, & Mitigation
Playback Data Privacy & Security
Playback Datasets, Integrations, & APIs
Last Mile Issue Detection, Mitigation, & Customer Messaging (Sonar)
Next-gen Continuous Optimization for Video Heuristics 
Playback Heuristic Treatment Generation, Promotion Service, & Player Simulator
Content & Context Aware Heuristics Optimization
Playback Optimization Algorithms & Science Initiatives
Playback Intake Request Mechanism Owner"

Answer: For analytics related to Playback, you should reach

In [32]:
prompt="What does the Live event and Linear Operations team do and who are the members of it?"
response=generate_answer(bedrock_rt, prompt, model_id, claude_config,system_prompt)
print(colored(response, "green"))

 based on the following relevant quotes from the document:

[1] "Live Event & Linear: The Live Event and Linear Product Teams owns the live events and linear product offerings for Playback. Our mission is to build and operate the world's most innovative live video streaming technology stack, providing a customer-centric live streaming experience. Our desire is to set the bar for the industry and position PV as the top streaming service in the world by being significantly better than broadcast TV in terms of availability, quality, reliability, and scale for live streaming."

[2] "Live Operations & Monitoring: The Live Operations and Live Monitoring Product teams owns the operation and end-to-end monitoring of all Live events and Linear stations on Prime Video. The program ensures we can rapidly onboard and operate events and linear stations at scale, including automation (all processes from pre-event to post-event), best-in-class 24/7 monitoring (finding defects before customers tell us

In [33]:
prompt="Who should I reach out for CDN Load Balancer?"
response=generate_answer(bedrock_rt, prompt, model_id, claude_config,system_prompt)
print(colored(response, "green"))

 Eric Boyd (byde@). Here are the relevant quotes:

[1] "Eric Boyd
byde@
Sr. Mgr Product
Sr. Manager Product, Playback Services and Video Delivery
Product lead for video delivery and play start services that deliver playable streams to customers world-wide
...
Next generation CDN Load Balancer (Project Serena)"

[2] "CDN edge authorization (Edge tokenization)
Granular Session Routing and optimized CDN URLs (UTS, GSR)
Next generation CDN Load Balancer (Project Serena)"

Answer: You should reach out to Eric Boyd (byde@) for questions or issues related to the CDN Load Balancer, specifically the "Next generation CDN Load Balancer (Project Serena)". Eric is the Sr. Manager of Product for Playback Services and Video Delivery, which includes the CDN Load Balancer product area. [1,2]
